# ETL

In [1]:
import pandas as pd
import hashlib
import unidecode
from datetime import datetime

filename = "soujunior_content_1717580556486.xls"
sheets = ["Métricas", "Todas as publicações"]

df_metricas = pd.read_excel(
    "files/" + filename,
    dtype=str,
    skiprows=1,
    sheet_name=sheets[0],
)

df_publicacoes = pd.read_excel(
    "files/" + filename,
    dtype=str,
    skiprows=1,
    sheet_name=sheets[1],
)

In [2]:
def generate_hash_id(*args):
    row_str = "".join(args)
    return hashlib.sha256(row_str.encode()).hexdigest()


def normalize_string(s):
    return (
        unidecode.unidecode(s)
        .lower()
        .replace(" ", "_")
        .replace("(", "")
        .replace(")", "")
    )


def etl(df):
    columns_name = [normalize_string(col) for col in df.columns]
    df.columns = columns_name
    df = df.fillna("0")
    df["id"] = df.apply(lambda row: generate_hash_id(str(row)), axis=1)
    df["origem"] = filename
    df["timestamp_ingestao"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    df["data_competencia"] = datetime.now().strftime("%Y-%m-01")
    df["fonte"] = "linkedin"
    return df

In [3]:
df_metricas = etl(df_metricas)
df_publicacoes = etl(df_publicacoes)

# Ingest

In [4]:
from sqlalchemy import create_engine, text
import os
from dotenv import load_dotenv

load_dotenv()

username = os.getenv("DB_USERNAME")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
database = os.getenv("DB_DATABASE")

engine = create_engine(
    f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}"
)

with engine.connect() as conn:
    result = conn.execute(text("SELECT 1"))
    print(result.scalar())

1


In [ ]:
df_publicacoes[df_publicacoes.columns.to_list()[5:]]

,criacao,data_de_inicio_da_campanha,data_de_termino_da_campanha,publico,impressoes,visualizacoes_excluindo_visualizacoes_de_video_fora_do_site,visualizacoes_fora_do_site,cliques,taxa_de_cliques_ctr,gostaram,comentarios,compartilhamentos,seguidores,taxa_de_engajamento,tipo_de_conteudo,id,origem,timestamp_ingestao,data_competencia,fonte
0,03/28/2024,0,0,Todos os seguidores,2386,416,0,43,0.01802179403603077,10,1,0,0,0.022632019594311714,Vídeo,ba3f66cc8ba2f5ebdbc7918c0bd62e77ca9b3ef92917df...,soujunior_content_1717580556486.xls,2025-01-14 20:47:35,2025-01-01,linkedin
1,03/27/2024,0,0,Todos os seguidores,2561,0,0,61,0.02381882071495056,25,2,2,0,0.03514252230525017,0,7535f6d39b7d9cb40b2ecd4b6d2c1f76d6f0d0a0d3840e...,soujunior_content_1717580556486.xls,2025-01-14 20:47:35,2025-01-01,linkedin
2,03/27/2024,0,0,Todos os seguidores,1860,0,0,44,0.023655913770198822,9,1,0,0,0.029032258316874504,0,5826e28e4391d2daec284f27bb68915ddedbc91f438c07...,soujunior_content_1717580556486.xls,2025-01-14 20:47:35,2025-01-01,linkedin
3,03/26/2024,0,0,Todos os seguidores,35918,0,0,1479,0.04117712751030922,103,33,2,0,0.04501920938491821,0,8fbb706c6129634debe6d398b159e4819f59cfd518ddd4...,soujunior_content_1717580556486.xls,2025-01-14 20:47:35,2025-01-01,linkedin
4,03/14/2024,0,0,Todos os seguidores,5510,1669,0,160,0.029038112610578537,58,103,6,0,0.05934664234519005,Vídeo,0d603c10a2d6cdd3732a33793df504c82f3c2cd0fa67d4...,soujunior_content_1717580556486.xls,2025-01-14 20:47:35,2025-01-01,linkedin
5,03/21/2024,0,0,Todos os seguidores,941,0,0,20,0.021253984421491623,2,0,0,0,0.023379383608698845,0,5734c5b4f8b00ffbcd43138f9a95bc4acd802ab1168ae1...,soujunior_content_1717580556486.xls,2025-01-14 20:47:35,2025-01-01,linkedin
6,03/20/2024,0,0,Todos os seguidores,3011,1494,0,73,0.024244436994194984,39,1,2,0,0.038193292915821075,Vídeo,89a59573ee20e831f8b360b7972aa856d2eb9b3e83f3d1...,soujunior_content_1717580556486.xls,2025-01-14 20:47:35,2025-01-01,linkedin
7,03/19/2024,0,0,Todos os seguidores,4500,1732,0,130,0.028888888657093048,46,1,3,0,0.03999999910593033,Vídeo,ee129621f9ff1e9d6f1da857887452d5b34de68e1f0912...,soujunior_content_1717580556486.xls,2025-01-14 20:47:35,2025-01-01,linkedin
8,03/19/2024,0,0,Todos os seguidores,4136,965,0,43,0.010396518744528294,35,14,0,0,0.022243713960051537,Vídeo,302af58f7f7f1077f3b001409401564b569723811e0345...,soujunior_content_1717580556486.xls,2025-01-14 20:47:35,2025-01-01,linkedin
9,03/19/2024,0,0,Todos os seguidores,20084,0,0,1832,0.09121689200401306,127,6,2,0,0.0979386568069458,0,65f25026589fb3074a54b18b954144cb2affc373a10969...,soujunior_content_1717580556486.xls,2025-01-14 20:47:35,2025-01-01,linkedin


In [ ]:
def convert_date_columns(df, columns):
    for column in columns:
        df[column] = pd.to_datetime(df[column], errors="coerce").dt.strftime("%Y-%m-%d").fillna("1970-01-01")
    return df

df_metricas = convert_date_columns(df_metricas, ["data"])
df_publicacoes = convert_date_columns(df_publicacoes, ["criacao", "data_de_inicio_da_campanha", "data_de_termino_da_campanha"])

/tmp/ipykernel_50939/1784400426.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[column] = pd.to_datetime(df[column], errors="coerce").dt.strftime("%Y-%m-%d").fillna("1970-01-01")
/tmp/ipykernel_50939/1784400426.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[column] = pd.to_datetime(df[column], errors="coerce").dt.strftime("%Y-%m-%d").fillna("1970-01-01")


In [7]:
query_metrics = """
CREATE TABLE IF NOT EXISTS linkedin_conteudo_metricas (
    id VARCHAR(255) PRIMARY KEY,
    timestamp_ingestao TIMESTAMP,
    data_competencia DATE,
    origem TEXT,
    fonte TEXT,
    data DATE,
    impressoes_organicas INTEGER,
    impressoes_patrocinadas INTEGER,
    impressoes_total INTEGER,
    impressoes_unicas_organicas INTEGER,
    cliques_organicos INTEGER,
    cliques_patrocinados INTEGER,
    cliques_total INTEGER,
    reacoes_organicas INTEGER,
    reacoes_patrocinadas INTEGER,
    reacoes_total INTEGER,
    comentarios_organicos INTEGER,
    comentarios_patrocinados INTEGER,
    comentarios_total INTEGER,
    compartilhamentos_organicos INTEGER,
    compartilhamentos_patrocinados INTEGER,
    compartilhamentos_total INTEGER,
    taxa_de_engajamento_organico FLOAT,
    taxa_de_engajamento_patrocinado FLOAT,
    taxa_de_engajamento_total FLOAT
);
"""

query_content = """
CREATE TABLE IF NOT EXISTS linkedin_conteudo_todas_as_publicacoes (
    id VARCHAR(255) PRIMARY KEY,
    timestamp_ingestao TIMESTAMP,
    data_competencia DATE,
    origem TEXT,
    fonte TEXT,
    titulo_da_publicacao TEXT,
    link_da_publicacao TEXT,
    tipo_de_publicacao TEXT,
    nome_da_campanha TEXT,
    publicada_por TEXT,
    criacao DATE,
    data_de_inicio_da_campanha DATE,
    data_de_termino_da_campanha DATE,
    publico TEXT,
    impressoes INTEGER,
    visualizacoes_excluindo_visualizacoes_de_video_fora_do_site INTEGER,
    visualizacoes_fora_do_site INTEGER,
    cliques INTEGER,
    taxa_de_cliques_ctr FLOAT,
    gostaram INTEGER,
    comentarios INTEGER,
    compartilhamentos INTEGER,
    seguidores INTEGER,
    taxa_de_engajamento FLOAT,
    tipo_de_conteudo TEXT
);
"""


def create_table(query):
    try:
        with engine.connect() as conn:
            conn.execute(text(query))
            print("tabela criada")

    except Exception as e:
        print("erro ao criar tabela")
        print(e)


create_table(query_metrics)
create_table(query_content)

tabela criada
tabela criada


In [8]:
def ingest_data(df, table_name):
    try:
        df.to_sql(
            table_name,
            con=engine,
            if_exists="append",
            index=False,
        )
        print("Dados inseridos com sucesso!")
    except Exception as e:
        print(e)


ingest_data(df_metricas, "linkedin_conteudo_metricas")
ingest_data(df_publicacoes, "linkedin_conteudo_todas_as_publicacoes")

Dados inseridos com sucesso!
Dados inseridos com sucesso!
